# SETUP

In [1]:
! pip install -q opensmile
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.3/494.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.6 MB/s eta 0:00:00


In [2]:
import opensmile
import audiofile
import pandas as pd
import os

In [35]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [36]:
SRC_CLEAN = "drive/MyDrive/idlproj/data/clean/"
SRC_NOISY = "drive/MyDrive/idlproj/data/auto/"
SRC_ENHAN = "drive/MyDrive/idlproj/data/enhan_auto_fullsubnet/enhanced_0010/"

In [ ]:
#required to be created if files have to be renamed in the noisy and enhanced directory

clean_file_map = dict()
noisy_file_map = dict()
enhan_file_map = dict()

for fn in os.listdir(SRC_CLEAN):
    identifier = fn.split("_")[-1].split(".")[0]
    clean_file_map[identifier] = fn

for fn in os.listdir(SRC_NOISY):
    identifier = fn.split("_")[-1].split(".")[0]
    noisy_file_map[identifier] = fn

for fn in os.listdir(SRC_ENHAN):
    identifier = fn.split("_")[-1].split(".")[0]
    enhan_file_map[identifier] = fn

In [13]:
# Loop over the dictionary and rename the files
for file_id, old_filename in enhan_file_map.items():
    # Construct the old filepath
    old_filepath = os.path.join(SRC_ENHAN, old_filename)

    # Get the new filename
    new_filename = 'enhan_' + file_id + '.wav'

    # Construct the new filepath
    new_filepath = os.path.join(SRC_ENHAN, new_filename)

    # Rename the file
    os.rename(old_filepath, new_filepath)

In [40]:
# Loop over the dictionary and rename the files
for file_id, old_filename in noisy_file_map.items():

    # Construct the old filepath
    old_filepath = os.path.join(SRC_NOISY, old_filename)

    # Get the new filename
    new_filename = 'auto_' + file_id + '.wav'

    # Construct the new filepath
    new_filepath = os.path.join(SRC_NOISY, new_filename)

    # Rename the file
    os.rename(old_filepath, new_filepath)

# Calculate Functionals

In [42]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
)
functionals_cols = smile.feature_names
functionals = pd.DataFrame(columns = functionals_cols)

In [43]:
print (functionals_cols)

['F0semitoneFrom27.5Hz_sma3nz_amean', 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm', 'F0semitoneFrom27.5Hz_sma3nz_percentile20.0', 'F0semitoneFrom27.5Hz_sma3nz_percentile50.0', 'F0semitoneFrom27.5Hz_sma3nz_percentile80.0', 'F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2', 'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope', 'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope', 'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope', 'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope', 'loudness_sma3_amean', 'loudness_sma3_stddevNorm', 'loudness_sma3_percentile20.0', 'loudness_sma3_percentile50.0', 'loudness_sma3_percentile80.0', 'loudness_sma3_pctlrange0-2', 'loudness_sma3_meanRisingSlope', 'loudness_sma3_stddevRisingSlope', 'loudness_sma3_meanFallingSlope', 'loudness_sma3_stddevFallingSlope', 'spectralFlux_sma3_amean', 'spectralFlux_sma3_stddevNorm', 'mfcc1_sma3_amean', 'mfcc1_sma3_stddevNorm', 'mfcc2_sma3_amean', 'mfcc2_sma3_stddevNorm', 'mfcc3_sma3_amean', 'mfcc3_sma3_stddevNorm', 'mfcc4_sma3_amean', 'mfcc4_sma3_std

In [44]:
from tqdm import tqdm 

def get_functionals(dir):  
    functionals = pd.DataFrame(columns = functionals_cols)
    for file in tqdm(sorted(os.listdir(dir))):
        if not file.endswith(".wav"):
          continue
        signal, sampling_rate = audiofile.read(dir + file,
          duration=10,
          always_2d=True)
        
        smile = opensmile.Smile(
          feature_set=opensmile.FeatureSet.eGeMAPSv02,
          feature_level=opensmile.FeatureLevel.Functionals,
        )
        
        res = smile.process_signal(
          signal,
          sampling_rate
        )
        res['FileId'] = file

        functionals = pd.concat([functionals, res])
    
    cols = functionals.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    functionals = functionals[cols]

    return functionals

In [45]:
clean_functionals = get_functionals(SRC_CLEAN)
noisy_functionals = get_functionals(SRC_NOISY)
enhan_functionals = get_functionals(SRC_ENHAN)


100%|██████████| 150/150 [02:38<00:00,  1.06s/it]


In [47]:
clean_functionals.to_csv("./drive/MyDrive/idlproj/features/clean_functionals.csv")
noisy_functionals.to_csv("./drive/MyDrive/idlproj/features/auto_functionals.csv")
enhan_functionals.to_csv("./drive/MyDrive/idlproj/features/enhan_auto_functionals.csv")

# Calculate Low level Descriptors

In [50]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

lowlevel_cols = smile.feature_names
lowlevel = pd.DataFrame(columns = lowlevel_cols)

In [51]:
from tqdm import tqdm 

def get_lld(dir):
    lowlevel = pd.DataFrame(columns = lowlevel_cols)
    for file in tqdm(sorted(os.listdir(dir))):
        if not file.endswith(".wav"):
          continue
        signal, sampling_rate = audiofile.read(
          dir + file,
          duration=10,
          always_2d=True
        )
        
        smile = opensmile.Smile(
          feature_set=opensmile.FeatureSet.eGeMAPSv02,
          feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
        )
        
        res = smile.process_signal(
          signal,
          sampling_rate
        )
        res['FileId'] = file
        # print(res.shape)
        # break
        lowlevel = pd.concat([lowlevel, res])
        
    cols = lowlevel.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    lowlevel = lowlevel[cols]

    return lowlevel

In [52]:
clean_lld = get_lld(SRC_CLEAN)
noisy_lld = get_lld(SRC_NOISY)
enhan_lld = get_lld(SRC_ENHAN)

100%|██████████| 150/150 [02:46<00:00,  1.11s/it]


In [53]:
clean_lld.to_csv("./drive/MyDrive/idlproj/features/clean_lld.csv")
noisy_lld.to_csv("./drive/MyDrive/idlproj/features/auto_lld.csv")
enhan_lld.to_csv("./drive/MyDrive/idlproj/features/enhan_auto_lld.csv")

In [59]:
import pandas as pd

# read in data
enhan_lld = pd.read_csv("./drive/MyDrive/idlproj/features/enhan_auto_lld.csv"
        ).sort_values(['FileId','Unnamed: 0'] # sort by filename and timedelta
        #).sort_values(['Unnamed: 0']
        ).iloc[:,2:].to_numpy() # don't include index/timedelta/fileid columns


### clean ###
clean_lld = pd.read_csv("./drive/MyDrive/idlproj/features/clean_lld.csv"
        ).sort_values(['FileId','Unnamed: 0'] # sort by filename and timedelta
        #).sort_values(['Unnamed: 0']
        ).iloc[:,2:].to_numpy() # don't include index/timedelta/fileid columns


### noisy ###
noisy_lld = pd.read_csv("./drive/MyDrive/idlproj/features/auto_lld.csv"
        ).sort_values(['FileId','Unnamed: 0'] # sort by filename and timedelta
        #).sort_values(['Unnamed: 0']
        ).iloc[:,2:].to_numpy() # don't include index/timedelta/fileid columns

# read in data
enhan_functionals = pd.read_csv("./drive/MyDrive/idlproj/features/enhan_auto_functionals.csv"
        ).sort_values(['Unnamed: 0'] # sort by filename and timedelta
        ).iloc[:,2:].to_numpy() # don't include index/timedelta/fileid columns


### clean ###
clean_functionals = pd.read_csv("./drive/MyDrive/idlproj/features/clean_functionals.csv"
        ).sort_values(['Unnamed: 0'] # sort by filename and timedelta
        ).iloc[:,2:].to_numpy() # don't include index/timedelta/fileid columns


### noisy ###
noisy_functionals = pd.read_csv("./drive/MyDrive/idlproj/features/auto_functionals.csv"
        ).sort_values(['Unnamed: 0'] # sort by filename and timedelta
        ).iloc[:,2:].to_numpy() # don't include index/timedelta/fileid columns

In [60]:

def get_mu(data):
    return data.mean(axis=0)

def get_std(data):
    return data.std(axis=0)

def get_standardized(data):
    # standardize data i.e. X = (X-MU) / STD
    return (data-get_mu(data))/get_std(data)

# MAE = lambda x,y: np.mean(np.abs(x-y),axis=0) # edited from konan's func b/c input data shapes differ

import sklearn.metrics

MAE = sklearn.metrics.mean_absolute_error

MAE_lld_enhan = MAE(get_standardized(enhan_lld), get_standardized(clean_lld), multioutput='raw_values')
MAE_lld_noisy = MAE(get_standardized(noisy_lld), get_standardized(clean_lld), multioutput='raw_values')

MAE_lld_enhan_agg = MAE(get_standardized(enhan_lld), get_standardized(clean_lld))
MAE_lld_noisy_agg = MAE(get_standardized(noisy_lld), get_standardized(clean_lld))

# calculate improvement 
I = lambda mae_x, mae_y: (mae_y - mae_x) / mae_y

I_lld_enhan_noisy = 100 * I( MAE_lld_enhan , MAE_lld_noisy )

In [61]:
print (I_lld_enhan_noisy.shape)

(25,)


In [62]:
# average MAE numbers

print (MAE_lld_enhan_agg)
print (MAE_lld_noisy_agg)

0.7175584128313897
0.7104335404954505


In [63]:
# imports
import opensmile
import numpy as np
import plotly
import plotly.subplots
import plotly.graph_objects
from google.colab import drive
import pandas as pd

# get feature names for low level acoustic
LLD = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors)

LLD_FEATURE_NAMES = LLD.feature_names

def make_plot(I_lld_auto_noisy, filename):
  fig = plotly.subplots.make_subplots(rows=1, cols=1, horizontal_spacing=0.01)

  ORDER = np.argsort(I_lld_auto_noisy)
  FEATURES = [LLD_FEATURE_NAMES[i].split("_")[0] for i in ORDER]

  fig.append_trace(
      row=1, col=1,
      trace = plotly.graph_objects.Bar(
          y=FEATURES, 
          x=I_lld_auto_noisy[ORDER], 
          orientation='h', name=f"FSN {filename} Improvement Over Auto"))

  HEIGHT = 290
  WIDTH  = 370

  fig.update_yaxes(showticklabels=False, col=2)
  fig.update_layout(
      height = 3*HEIGHT, 
      width  = 3*WIDTH,
      legend=dict(orientation="h", yanchor="bottom"),
      margin = dict(l=0, r=0, t=0, b=0),
      bargap =0.50,
      # xaxis1_range=[-70,100],
      xaxis1_range=[I_lld_auto_noisy.min()-5,I_lld_auto_noisy.max()+5],
      xaxis1=dict(tickmode='linear', dtick=20)
  )

  FONT_FAMILY = "Times New Roman"
  FONT_SIZE   = 9

  keys = list(locals().keys())
  for l in keys:
      if l[:3] == 'fig':
          locals()[l].update_layout(font_family=FONT_FAMILY, font_size = 2*FONT_SIZE)

  fig.show()

  fig.write_image("drive/MyDrive/idlproj/Auto LLD Improvment.png")

In [ ]:
make_plot(I_lld_enhan_noisy, "LLD")

In [65]:

def get_mu(data):
    return data.mean(axis=0)

def get_std(data):
    return data.std(axis=0)

def get_standardized(data):
    # standardize data i.e. X = (X-MU) / STD
    return (data-get_mu(data))/get_std(data)

# MAE = lambda x,y: np.mean(np.abs(x-y),axis=0) # edited from konan's func b/c input data shapes differ

import sklearn.metrics

MAE = sklearn.metrics.mean_absolute_error

MAE_functionals_enhan = MAE(get_standardized(enhan_functionals), get_standardized(clean_functionals), multioutput='raw_values')
MAE_functionals_noisy = MAE(get_standardized(noisy_functionals), get_standardized(clean_functionals), multioutput='raw_values')

MAE_functionals_enhan_agg = MAE(get_standardized(enhan_functionals), get_standardized(clean_functionals))
MAE_functionals_noisy_agg = MAE(get_standardized(noisy_functionals), get_standardized(clean_functionals))

# calculate improvement 
I = lambda mae_x, mae_y: (mae_y - mae_x) / mae_y

I_functionals_enhan_noisy = 100 * I( MAE_functionals_enhan , MAE_functionals_noisy )

In [ ]:
print (I_functionals_enhan_noisy.shape)

In [67]:
# average MAE numbers

print (MAE_functionals_enhan_agg)
print (MAE_functionals_noisy_agg)

0.7429446566017095
0.6967222054562329


In [68]:
# imports
import opensmile
import numpy as np
import plotly
import plotly.subplots
import plotly.graph_objects
from google.colab import drive
import pandas as pd

# get feature names for low level acoustic
functionals = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals)

FUNCTIONALS_FEATURE_NAMES = functionals.feature_names

def make_plot(I_functionals_auto_noisy, filename):
  fig = plotly.subplots.make_subplots(rows=1, cols=1, horizontal_spacing=0.01)

  ORDER = np.argsort(I_functionals_auto_noisy)
  #FEATURES = [FUNCTIONALS_FEATURE_NAMES[i].split("_")[0] for i in ORDER]
  FEATURES = [FUNCTIONALS_FEATURE_NAMES[i] for i in ORDER]

  print (FEATURES)

  fig.append_trace(
      row=1, col=1,
      trace = plotly.graph_objects.Bar(
          y=FEATURES, 
          x=I_functionals_auto_noisy[ORDER], 
          orientation='h', name=f"FSN {filename} Improvement Over Auto"))

  HEIGHT = 400
  WIDTH  = 370

  fig.update_yaxes(showticklabels=True, col=2)
  fig.update_layout(
      height = 3*HEIGHT, 
      width  = 3*WIDTH,
      legend=dict(orientation="h", yanchor="bottom"),
      margin = dict(l=0, r=0, t=0, b=0),
      bargap =0.50,
      # xaxis1_range=[-70,100],
      xaxis1_range=[I_functionals_auto_noisy.min()-5,I_functionals_auto_noisy.max()+5],
      xaxis1=dict(tickmode='linear', dtick=20)
  )

  FONT_FAMILY = "Times New Roman"
  FONT_SIZE   = 4

  keys = list(locals().keys())
  for l in keys:
      if l[:3] == 'fig':
          locals()[l].update_layout(font_family=FONT_FAMILY, font_size = 2*FONT_SIZE)

  fig.show()

  fig.write_image("drive/MyDrive/idlproj/Auto Functionals Improvement.png")

In [ ]:
make_plot(I_functionals_enhan_noisy, "Functionals")